In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from hera_cal import redcal
import linsolve

In [2]:
Nants = 37

ants = np.loadtxt('antenna_positions_%d.dat'%Nants)
antpos = {k:v for k,v in zip(range(Nants),ants)}
reds = redcal.get_reds(antpos)

In [3]:
def calibrate(data,gains):
    calib_data = {}
    for (i, j, pol) in data.keys():
        calib_data[(i, j, pol)] = data[(i, j, pol)]/(gains[(i,'Jxx')] * np.conj(gains[(j,'Jxx')]))
    return calib_data

def average_red_vis(data, gains, subreds):
    sub_bl_visib = calibrate(data, gains)
    vis = 0
    for subbl in subreds:
        vis += sub_bl_visib[subbl]/len(subreds)
    return vis

def optimize_red_vis(data, gains, subreds):
    eqs = {}; const = {}
    for (i,j,pol) in subreds:
        eqs['g_%d * g_%d_ * V'%(i,j)] = data[(i,j,pol)]

    for i in range(37):
        const['g_%d'%(i)] = gains[(i,'Jxx')]    

    lps = linsolve.LinearSolver(eqs, **const)
    X = lps.solve()
    return X['V']

1. Case1  all: snr > 1

2. Case2  all: snr < 1

3. snr ~ 1 higher integration and snr < 1 lower integration 

90 t_0 = 666 t_1

sigma = N/np.sqrt(t)  

Y = Vtrue + sigma

snr = Vtrue/sigma

In [4]:
Nsim = 64

# Simulate gains and visibilities
true_gains, true_vis, true_data = redcal.sim_red_data(reds, shape=(1, Nsim), gain_scatter=0.1)

In [5]:
subredbls = reds[0:3]
subvis_keys = [(0,1,'xx'),(0,4,'xx'),(0,5,'xx')]

# SNR > 1

In [6]:
sigma_sub = 0.1
sigma_low = 0.1 * np.sqrt(666/90.)

data_sub = {k:v+ sigma_sub*redcal.noise((1, Nsim)) for k,v in true_data.items()}
data_low = {k:v+ sigma_low*redcal.noise((1, Nsim)) for k,v in true_data.items()}
    
redcalibrator = redcal.RedundantCalibrator(reds)
sol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
sol_degen.update({k:np.ones_like(v) for k,v in true_vis.items()})

sol_degen = redcalibrator.omnical(data_low, sol_degen, gain= 0.4, maxiter= 1000, check_after=10)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

redcalibrator_sub = redcal.RedundantCalibrator(subredbls)
subsol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
subsol_degen.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subvis_keys})

subsol_degen = redcalibrator_sub.omnical(data_sub, subsol_degen, gain= 0.4, maxiter= 1000, check_after=10)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
# for k,v in data_sub.items():
#     print np.any(np.isnan(v)),
#     print np.any(v == 0)

In [7]:
#sol_degen
# print "No \t chisq \t conv_crit\t iter"
# print "-----------------------------------------"
# for n in range(Nsim):
#     print "%2d\t %4.2f \t %1.4e\t %1.4e"%(n, sol_degen[0]['chisq'][0,n], sol_degen[0]['conv_crit'][0,n], 
#                                           sol_degen[0]['iter'][0,n])
sol_degen[0]

{'chisq': array([[39.83866988, 41.48363244, 40.17138676, 43.55817956, 41.91269344,
         42.81220762, 40.08027408, 41.13879643, 42.52848291, 43.3114115 ,
         41.26454204, 42.30063341, 42.66532983, 37.73351717, 41.67572223,
         42.70157696, 43.9649879 , 38.79614594, 42.45697056, 39.7078803 ,
         43.49065421, 40.86094803, 42.81804536, 42.19027153, 40.30028588,
         40.9376055 , 42.00852335, 40.95847324, 42.45484669, 41.10590442,
         42.13801333, 40.72683728, 42.63606509, 39.39697384, 42.11533002,
         43.32092608, 41.22667404, 42.07870119, 40.34168312, 42.64789621,
         44.49647471, 43.40975257, 39.18135641, 41.69041018, 43.08971782,
         42.92566186, 40.88125407, 42.57382379, 40.32902062, 40.12463441,
         42.41831423, 42.46022877, 41.64984913, 38.54618435, 40.90847366,
         40.23044544, 42.52935639, 44.46749652, 42.46029809, 44.08197481,
         42.75854829, 42.7853461 , 41.29052859, 41.64358685]]),
 'conv_crit': array([[7.36352764e-10, 5

In [8]:
# subsol_degen
# print "No \t chisq \t conv_crit\t iter"
# print "-----------------------------------------"
# for n in range(Nsim):
#     print "%2d\t %4.2f \t %1.2e\t %1.2e"%(n, subsol_degen[0]['chisq'][0,n], subsol_degen[0]['conv_crit'][0,n], 
#                                           subsol_degen[0]['iter'][0,n])
subsol_degen[0]

{'chisq': array([[0.57441293, 0.52916691, 0.58256837, 0.61501747, 0.43383035,
         0.52916003, 0.50071744, 0.48715586, 0.47441882, 0.5111112 ,
         0.52205899, 0.50557901, 0.51114355, 0.64848912, 0.58348518,
         0.41210892, 0.46874218, 0.57533792, 0.40880863, 0.51594231,
         0.45558902, 0.52291263, 0.47857584, 0.49188519, 0.5120296 ,
         0.46377868, 0.46019303, 0.36065572, 0.56930924, 0.44677126,
         0.44784549, 0.45998646, 0.6223468 , 0.31185797, 0.7350947 ,
         0.47762456, 0.5863073 , 0.49968336, 0.59238997, 0.5330418 ,
         0.47387681, 0.57615378, 0.48903128, 0.42244916, 0.5744694 ,
         0.63415333, 0.544606  , 0.45889868, 0.53774316, 0.52581935,
         0.37155669, 0.56612203, 0.42858972, 0.45793175, 0.42891522,
         0.6068641 , 0.52484935, 0.44131961, 0.52607992, 0.51266953,
         0.536747  , 0.53178809, 0.56090886, 0.48558264]]),
 'conv_crit': array([[1.62657597e-10, 1.80565178e-10, 3.10929073e-10, 1.56724597e-06,
         7.307749

In [ ]:
# Check chisq of just the shortest baselines
chisq = 0
for k in subvis_keys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data_sub[(i,j,pol)])**2
        
chisq = chisq/((90-40+2)*sigma_sub**2)
print np.mean(chisq[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)

In [ ]:
chisq_sub = 0; chisq_low = 0; chisq_sub_3bl = 0

for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0] 
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2
        chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2
        if k in subvis_keys:
            chisq_sub_3bl += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*subsol[k] -data_sub[(i,j,pol)]))**2
        
chisq_sub = chisq_sub/((666-100+2)*sigma_sub**2)
chisq_low = chisq_low/((666-100+2)*sigma_low**2)

print "SUBREDCAL: mean: ", np.mean(chisq_sub[0]), " median: ", np.median(chisq_sub[0])
print "LOWCADCAL: mean: ", np.mean(chisq_low[0]), " median: ", np.median(chisq_low[0])

# Subredcal chi^2 for the baselines included only
print "subredcal: only included baselines: ", np.mean(chisq_sub_3bl[0])/((90-40+2)*sigma_sub**2)

# SNR < 1

In [ ]:
sigma_sub = 2
sigma_low = sigma_sub * np.sqrt(666/90.)

data_sub = {k:v+ sigma_sub*redcal.noise((1, Nsim)) for k,v in true_data.items()}
data_low = {k:v+ sigma_low*redcal.noise((1, Nsim)) for k,v in true_data.items()}
    
## Can't guarantee that in low SNR logcal is within a phase wrap
## lincal -> omnical
## const to LinProductSolver -- if you give it equations it assumes it can change those values

    
redcalibrator = redcal.RedundantCalibrator(reds)
#sol_degen = redcalibrator.logcal(data_low)
sol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
sol_degen.update({k:np.ones_like(v) for k,v in true_vis.items()})

sol_degen = redcalibrator.omnical(data_low, sol_degen, gain= 0.4, maxiter= 5000, 
                                  check_after= 50)#, conv_crit= 1e-5)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

redcalibrator_sub = redcal.RedundantCalibrator(subredbls)
#subsol_degen = redcalibrator_sub.logcal(data_sub)
subsol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
subsol_degen.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subvis_keys})
subsol_degen = redcalibrator_sub.omnical(data_sub, subsol_degen, gain= 0.4, maxiter= 5000, 
                                         check_after= 50)#, conv_crit= 1e-5)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
#sol_degen
print "No \t chisq \t conv_crit\t iter"
print "-----------------------------------------"
for n in range(Nsim):
    print "%2d\t %4.2f \t %1.2e\t %1.2e"%(n, sol_degen[0]['chisq'][0,n], sol_degen[0]['conv_crit'][0,n], 
                                          sol_degen[0]['iter'][0,n])

In [ ]:
# subsol_degen
print "No \t chisq \t conv_crit\t iter"
print "-----------------------------------------"
for n in range(Nsim):
    print "%2d\t %4.2f \t %1.2e\t %1.2e"%(n, subsol_degen[0]['chisq'][0,n], subsol_degen[0]['conv_crit'][0,n], 
                                          subsol_degen[0]['iter'][0,n])

In [ ]:
# Check chisq of just the shortest baselines
chisq = 0
for k in subvis_keys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data_sub[(i,j,pol)])**2
        
chisq = chisq/((90-40+2)*sigma_sub**2)
print np.mean(chisq[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)

In [ ]:
chisq_sub = 0; chisq_low = 0; chisq_sub_3bl=0;

for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2
        chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2
        if k in subvis_keys:
            chisq_sub_3bl += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*subsol[k] -data_sub[(i,j,pol)]))**2
        
chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)
chisq_low = chisq_low/((666-100+4)*sigma_low**2)

print "SUBREDCAL: mean: ", np.mean(chisq_sub[0]), " median: ", np.median(chisq_sub[0])
print "LOWCADCAL: mean: ", np.mean(chisq_low[0]), " median: ", np.median(chisq_low[0])

# Subredcal chi^2 for the baselines included only
chisq_sub_3bl = chisq_sub_3bl/((90-40+4)*sigma_sub**2)

print "subredcal: only included baselines: ", np.mean(chisq_sub_3bl[0])

# Border case

In [ ]:
sigma_sub = 0.4
sigma_low = sigma_sub * np.sqrt(666/90.)

data_sub = {k:v+ sigma_sub*redcal.noise((1, Nsim)) for k,v in true_data.items()}
data_low = {k:v+ sigma_low*redcal.noise((1, Nsim)) for k,v in true_data.items()}
    
redcalibrator = redcal.RedundantCalibrator(reds)
#sol_degen = redcalibrator.logcal(data_low)
sol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
sol_degen.update({k:np.ones_like(v) for k,v in true_vis.items()})
sol_degen = redcalibrator.omnical(data_low, sol_degen, gain=0.2, maxiter=1000,
                                 check_after= 10, conv_crit=1e-6)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

redcalibrator_sub = redcal.RedundantCalibrator([reds[0],reds[1],reds[2]])
#subsol_degen = redcalibrator_sub.logcal(data_sub)
subsol_degen = {k:np.ones_like(v) for k,v in true_gains.items()}
subsol_degen.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subvis_keys})
subsol_degen = redcalibrator_sub.omnical(data_sub, subsol_degen, gain=0.2, maxiter=1000, 
                                         check_after= 10, conv_crit=1e-6)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
#sol_degen
print "No \t chisq \t conv_crit\t iter"
print "-----------------------------------------"
for n in range(Nsim):
    print "%2d\t %4.2f \t %1.2e\t %1.2e"%(n, sol_degen[0]['chisq'][0,n], sol_degen[0]['conv_crit'][0,n], 
                                          sol_degen[0]['iter'][0,n])

In [ ]:
# subsol_degen
print "No \t chisq \t conv_crit\t iter"
print "-----------------------------------------"
for n in range(Nsim):
    print "%2d\t %4.2f \t %1.2e\t %1.2e"%(n, subsol_degen[0]['chisq'][0,n], subsol_degen[0]['conv_crit'][0,n], 
                                          subsol_degen[0]['iter'][0,n])

In [ ]:
# Check chisq of just the shortest baselines
chisq = 0
for k in subvis_keys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data_sub[(i,j,pol)])**2
        
chisq = chisq/((90-40+4)*sigma_sub**2)
print chisq[0]

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)

In [ ]:
chisq_sub = 0; chisq_low = 0; chisq_sub_3bl=0;

for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2
        chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2
        if k in subvis_keys:
            chisq_sub_3bl += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*subsol[k] -data_sub[(i,j,pol)]))**2
        
chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)
chisq_low = chisq_low/((666-100+4)*sigma_low**2)

print "SUBREDCAL: mean: ", np.mean(chisq_sub[0]), " median: ", np.median(chisq_sub[0])
print "LOWCADCAL: mean: ", np.mean(chisq_low[0]), " median: ", np.median(chisq_low[0])

# Subredcal chi^2 for the baselines included only
chisq_sub_3bl = chisq_sub_3bl/((90-40+4)*sigma_sub**2)

print "subredcal: only included baselines: ", np.mean(chisq_sub_3bl[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)

In [ ]:
chisq_sub = 0; chisq_low = 0; chisq_sub_3bl=0;

for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2
        chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2
        if k in subvis_keys:
            chisq_sub_3bl += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*subsol[k] -data_sub[(i,j,pol)]))**2
        
chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)
chisq_low = chisq_low/((666-100+4)*sigma_low**2)

print "SUBREDCAL: mean: ", np.mean(chisq_sub[0]), " median: ", np.median(chisq_sub[0])
print "LOWCADCAL: mean: ", np.mean(chisq_low[0]), " median: ", np.median(chisq_low[0])

# Subredcal chi^2 for the baselines included only
chisq_sub_3bl = chisq_sub_3bl/((90-40+4)*sigma_sub**2)

print "subredcal: only included baselines: ", np.mean(chisq_sub_3bl[0])

In [ ]:
subgain = []
allgain = []

for ant in range(37):
    allgain.append(np.log(sol[(ant,'Jxx')][0]))
    subgain.append(np.log(subsol[(ant,'Jxx')][0]))

subcovr = np.cov(np.real(subgain))
subcovi = np.cov(np.imag(subgain))

allcovr = np.cov(np.real(allgain))
allcovi = np.cov(np.imag(allgain))

plt.figure(figsize=(8,8))
plt.subplot(221)
plt.title('Real All')
plt.imshow(np.abs(allcovr))
plt.colorbar(shrink=0.7)

plt.subplot(222)
plt.title('Real Subset')
plt.imshow(np.abs(subcovr))
plt.colorbar(shrink=0.7)

plt.subplot(223)
plt.title('Imag all')
plt.imshow(np.abs(allcovi))
plt.colorbar(shrink=0.7)

plt.subplot(224)
plt.title('Imag subset')
plt.imshow(np.abs(subcovi))
plt.colorbar(shrink=0.7)

In [ ]:
chisq_sub = 0; chisq_low = 0

for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0] 
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2
        chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2
        
chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)
chisq_low = chisq_low/((666-100+4)*sigma_low**2)


print "SUBREDCAL: mean: ", np.mean(chisq_sub[0]), " median: ", np.median(chisq_sub[0])
print "LOWCADCAL: mean: ", np.mean(chisq_low[0]), " median: ", np.median(chisq_low[0])

In [ ]:
Nsim2 = int(np.log(Nsim)/np.log(2))

steps = np.logspace(0, Nsim2, base=2, num=Nsim2+1, dtype=np.int)
chisq_sub = np.zeros([Nsim2+1, Nsim])

model_vis_sub = {}

for n,i in enumerate(steps):
    print i
    # (i) Average the gains
    gain_sub = {}
    for k in true_gains.keys():
        gain_sub[k] = np.asarray([np.repeat(np.mean(subsol[k].reshape(-1,i), axis=1), i, axis=0)])

    # (ii) Compute all model visibilities with the averaged gains
    model_vis_sub[n] = {}
    
    for k in true_vis.keys():
        if k in subsol.keys():
            model_vis_sub[n][k] = subsol[k]
        else:
            subreds = [bls for bls in reds if k in bls][0]
            model_vis_sub[n][k] = optimize_red_vis(data_sub, gain_sub, subreds)

            
    # (iii) Compute the chisq with these new model visibilities
    for k in true_vis.keys():
        subreds = [bls for bls in reds if k in bls][0] 
        for (a1,a2,pol) in subreds:
            chisq_sub[n] += np.abs((gain_sub[(a1,'Jxx')]*np.conj(gain_sub[(a2,'Jxx')])\
                                    *model_vis_sub[n][k] -data_sub[(a1,a2,pol)])[0])**2

    chisq_sub[n] = chisq_sub[n]/((i*666. - (37+ (63*i)- (4*i)))* sigma_sub**2)

In [ ]:
plt.figure()
for n,i in enumerate(steps):
    chi = np.mean(np.sum(chisq_sub[n].reshape(-1,i),axis=1))
    print i, chi
    plt.plot(i, chi, 'o')

plt.plot(1,np.mean(chisq_low[0]),'o',color='k')
plt.grid()
plt.semilogx()
plt.xlabel('Number of simulations averaged over')
plt.ylabel('')

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(11,5),sharey=True)

dev_sub = np.asarray([]); 
dev_all = np.asarray([])
subsol_vis_keys = [(0,1,'xx'),(0,4,'xx'),(0,5,'xx')]

for k in true_vis.keys():
    subreds = [ _ for _ in reds if k in _ ][0]
    for subbl in subreds:
        dev_sub = np.append(dev_sub, np.abs(model_vis_sub[subbl]-true_vis[k])[0])
        dev_all = np.append(dev_all, np.abs(sol[subbl]-true_vis[k])[0])

n, b, p  = ax[0].hist(dev_sub, bins=256, color='k', alpha=0.5, label='subredcal')
na,ba,pa = ax[0].hist(dev_all, bins=b,  color='m', alpha=0.5, label='redcal')
ax[0].legend()
ax[0].set_title('Baselines in subredcal')
ax[0].set_xlabel(r'$\left|V^{calib}_{ij} - V^{true}_{ij} \right|$')

dev_sub = np.asarray([]); dev_all = np.asarray([])
for k in model_vis_sub.keys():
    if k not in subsol_vis_keys:
        subreds = [ _ for _ in reds if k in _ ][0]
        for subbl in subreds:
            dev_sub = np.append(dev_sub, np.abs(subreds_vis_sub[subbl]-true_vis[k])[0])
            dev_all = np.append(dev_all, np.abs(subreds_vis_all[subbl]-true_vis[k])[0])

n, b, p  = ax[1].hist(dev_sub, bins=3000, color='k', alpha=0.5, label='subredcal')
na,ba,pa = ax[1].hist(dev_all, bins=b,  color='m', alpha=0.5, label='redcal')
ax[1].legend()
ax[1].set_xlim([0,0.25])
ax[1].set_title('All other baselines')
ax[1].set_xlabel(r'$\left|V^{calib}_{ij} - V^{true}_{ij} \right|$')

# Minimum SNR for convergence

In [ ]:
sigma = np.linspace(0.01,0.5,num=11)
sigma = [0.4]

for sigma_sub in sigma:
    data_sub = {k:v+ sigma_sub*redcal.noise((1, Nsim)) for k,v in true_data.items()}

    redcalibrator_sub = redcal.RedundantCalibrator([reds[0],reds[1],reds[2]])
    subsol_degen = redcalibrator_sub.logcal(data_sub)
    subsol_degen = redcalibrator_sub.lincal(data_sub, subsol_degen)
    subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)
    
    # Optimize for the other visibilities
    model_vis_sub = {}
    for k in true_vis.keys():
        if k in subsol.keys():
            model_vis_sub[k] = subsol[k]
        else:
            subreds = [bls for bls in reds if k in bls][0]
            model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)
            
    # Compute chisq
    chisq_sub = 0
    
    for k in true_vis.keys():
        subreds = [bls for bls in reds if k in bls][0] 
        for (i,j,pol) in subreds:
            chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2

    chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)

    print sigma_sub, np.median(chisq_sub[0])

In [ ]:
sigma = np.linspace(0.01,0.5,num=11)


for sigma_sub in sigma:
    data_sub = {k:v+ sigma_sub*redcal.noise((1, Nsim)) for k,v in true_data.items()}

    redcalibrator_sub = redcal.RedundantCalibrator(reds[0:6])
    subsol_degen = redcalibrator_sub.logcal(data_sub)
    subsol_degen = redcalibrator_sub.lincal(data_sub, subsol_degen)
    subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)
    
    # Optimize for the other visibilities
    model_vis_sub = {}
    for k in true_vis.keys():
        if k in subsol.keys():
            model_vis_sub[k] = subsol[k]
        else:
            subreds = [bls for bls in reds if k in bls][0]
            model_vis_sub[k] = optimize_red_vis(data_sub, subsol, subreds)
            
    # Compute chisq
    chisq_sub = 0
    
    for k in true_vis.keys():
        subreds = [bls for bls in reds if k in bls][0] 
        for (i,j,pol) in subreds:
            chisq_sub += np.abs((subsol[i,'Jxx']*np.conj(subsol[j,'Jxx'])*model_vis_sub[k] -data_sub[(i,j,pol)]))**2

    chisq_sub = chisq_sub/((666-100+4)*sigma_sub**2)

    print sigma_sub, np.mean(chisq_sub[0])

# Low cadence - minimum SNR for convergence

In [ ]:
sigma = np.linspace(0.5,1.5,num=11)
sigma

In [ ]:
sigma = np.linspace(0.5,1.5,num=11)

for sigma_low in sigma:
    data_low = {k:v+ sigma_low*redcal.noise((1, Nsim)) for k,v in true_data.items()}

    redcalibrator = redcal.RedundantCalibrator(reds)
    sol_degen = redcalibrator.logcal(data_low)
    sol_degen = redcalibrator.lincal(data_low, sol_degen)
    sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)
            
    # Compute chisq
    chisq_low = 0
    
    for k in true_vis.keys():
        subreds = [bls for bls in reds if k in bls][0] 
        for (i,j,pol) in subreds:
            chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2

    chisq_low = chisq_low/((666-100+4)*sigma_low**2)

    print sigma_low, np.mean(chisq_low[0])

In [ ]:
sigma = np.linspace(2,2.5,num=11)

for sigma_low in sigma:
    data_low = {k:v+ sigma_low*redcal.noise((1, Nsim)) for k,v in true_data.items()}

    redcalibrator = redcal.RedundantCalibrator(reds)
    sol_degen = redcalibrator.logcal(data_low)
    sol_degen = redcalibrator.lincal(data_low, sol_degen)
    sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)
            
    # Compute chisq
    chisq_low = 0
    
    for k in true_vis.keys():
        subreds = [bls for bls in reds if k in bls][0] 
        for (i,j,pol) in subreds:
            chisq_low += np.abs((sol[i,'Jxx']*np.conj(sol[j,'Jxx'])*sol[k] -data_low[(i,j,pol)]))**2

    chisq_low = chisq_low/((666-100+4)*sigma_low**2)

    print sigma_low, np.mean(chisq_low[0])